In [ ]:
# %%capture --no-stderr
# %pip install -U tavily-python langchain_community


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os

TAVILY_API_KEY = "<TAVILY_API_KEY>"

os.environ["TAVILY_API_KEY"] = "config.json"